In [2]:
import time
import re
import pandas as pd
import os
from bs4 import BeautifulSoup
import requests
import numpy as np
import math

In [ ]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException, TimeoutException, StaleElementReferenceException, ElementClickInterceptedException

### Агрегированные данные по депутатам и партиям

In [3]:
# загрузка данных
dep_full_data = pd.read_csv('/content/dep_full_data.csv')

Топ-10 депутатов-прогульщиков, которые пропустили больше всего заседаний в этом созыве. 9 из 10 относятся к партии Единая Россия. А депутат Колесников пропустил почти половину заседаний. "Важные истории" ещё выяснили, что он за время этого созыва даже ни разу не выступил.

Можно отправить этим депутатам запросы и спросить, почему они пропустили так много заседаний. Изучить их законодательную деятельность и посмотреть собираются ли они идти на выборы. 

In [8]:
# топ-10 депутатов прогульщиков
top_abs_dep = dep_full_data.groupby(['name', 'party'])[['total_absence', 'meetings', 'percent_of_presence']].sum().sort_values(by='total_absence', ascending=False).reset_index()
top_abs_dep.columns = ['имя_депутата', 'партия', 'сколько_заседаний_пропустил', 'сколько_заседений_всего', 'процент_присутсвия']
top_abs_dep.head(10)

,имя_депутата,партия,сколько_заседаний_пропустил,сколько_заседений_всего,процент_присутсвия
0,Колесников Олег Алексеевич,"Фракция Всероссийской политической партии ""ЕДИ...",172.0,365,0.529
1,Барышев Андрей Викторович,"Фракция Всероссийской политической партии ""ЕДИ...",129.0,365,0.647
2,Саблин Дмитрий Вадимович,"Фракция Всероссийской политической партии ""ЕДИ...",127.0,365,0.652
3,Марченко Евгений Евгеньевич,"Фракция Всероссийской политической партии ""ЕДИ...",115.0,365,0.685
4,Ремезков Александр Александрович,Фракция Политической партии СПРАВЕДЛИВАЯ РОССИЯ,107.0,365,0.707
5,Жупиков Александр Владимирович,"Фракция Всероссийской политической партии ""ЕДИ...",103.0,362,0.715
6,Бурнашов Алексей Леонидович,"Фракция Всероссийской политической партии ""ЕДИ...",102.0,365,0.721
7,Мищеряков Юрий Николаевич,"Фракция Всероссийской политической партии ""ЕДИ...",98.0,365,0.732
8,Скриванов Дмитрий Станиславович,"Фракция Всероссийской политической партии ""ЕДИ...",96.0,365,0.737
9,Муцоев Зелимхан Аликоевич,"Фракция Всероссийской политической партии ""ЕДИ...",95.0,365,0.740


Это, наоборот,  депутаты-отличники, которые посетили либо все, либо почти все заседания

In [12]:
# топ-10 депутатов-отличников
top_pres_dep = dep_full_data.groupby(['name', 'party'])[['total_absence', 'meetings', 'percent_of_presence']].sum().sort_values(by='total_absence', ascending=True).reset_index()
top_pres_dep.columns = ['имя_депутата', 'партия', 'сколько_заседаний_пропустил', 'сколько_заседений_всего', 'процент_присутсвия']
top_pres_dep.head(10)

,имя_депутата,партия,сколько_заседаний_пропустил,сколько_заседений_всего,процент_присутсвия
0,Хасанов Мурат Русланович,"Фракция Всероссийской политической партии ""ЕДИ...",0.0,365,1.000
1,Куринный Алексей Владимирович,"Фракция Политической партии ""КОММУНИСТИЧЕСКАЯ ...",0.0,365,1.000
2,Альшевских Андрей Геннадьевич,"Фракция Всероссийской политической партии ""ЕДИ...",0.0,365,1.000
3,Гришин Евгений Анатольевич,"Фракция Всероссийской политической партии ""ЕДИ...",0.0,31,1.000
4,Калашников Сергей Вячеславович,Фракция Политической партии ЛДПР - Либерально-...,0.0,51,1.000
5,Волков Юрий Геннадьевич,Фракция Политической партии ЛДПР - Либерально-...,0.0,365,1.000
6,Минкин Иршат Султанович,"Фракция Всероссийской политической партии ""ЕДИ...",1.0,365,0.997
7,Щапов Михаил Викторович,"Фракция Политической партии ""КОММУНИСТИЧЕСКАЯ ...",1.0,365,0.997
8,Иванюженков Борис Викторович,"Фракция Политической партии ""КОММУНИСТИЧЕСКАЯ ...",1.0,91,0.989
9,Боярский Сергей Михайлович,"Фракция Всероссийской политической партии ""ЕДИ...",1.0,365,0.997


Несмотря на то, что главные прогульщики из "Единой России", но по общему числу прогулов эта партия на несколько процентов остаёт от "Справедливой России". Самыми же дисциплинированными оказались коммунисты.

In [9]:
# прогулы по партиям в процентах
party_abs = dep_full_data.groupby(['party'])[['total_absence', 'meetings']].sum().sort_values(by='total_absence', ascending=False)
party_abs_total = pd.DataFrame(party_abs.total_absence / party_abs.meetings * 100)
party_abs_total.rename(columns={0:'процент_прогулов'},  inplace=True)
party_abs_total = party_abs_total.rename_axis('партия')
party_abs_total.sort_values(by='процент_прогулов', ascending=False)

,процент_прогулов
партия,
Фракция Политической партии СПРАВЕДЛИВАЯ РОССИЯ,11.194133
"Фракция Всероссийской политической партии ""ЕДИНАЯ РОССИЯ""",7.843605
Фракция Политической партии ЛДПР - Либерально-демократической партии России,3.612755
"Депутаты, не входящие во фракции",3.576341
"Фракция Политической партии ""КОММУНИСТИЧЕСКАЯ ПАРТИЯ РОССИЙСКОЙ ФЕДЕРАЦИИ""",2.876359


Всего же депутаты прогуляли чуть больше 7% заседаний

In [10]:
# общий процент прогулов
dep_full_data.total_absence.sum(axis=0) / dep_full_data.meetings.sum(axis=0)  * 100

7.186336464560204

Ещё по хранящимся на депутатов данных можно понять как они голосовали на заседаниях. Далее топ-10 депутатов, которые  чаще всего голосуют "За". Других партий кроме "Единой России" в этом списке нет.

In [11]:
# топ-10 депутатов, которые чаще голосовали "за"
top_yes_dep = dep_full_data.groupby(['name', 'party'])[['voting_yes_count', 'voting_yes']].sum().sort_values(by='voting_yes_count', ascending=False).reset_index()
top_yes_dep.columns = ['имя_депутата', 'партия', 'всего_голосов_за', 'процент_голосов_за']
top_yes_dep.head(10)

,имя_депутата,партия,всего_голосов_за,процент_голосов_за
0,Хасанов Мурат Русланович,"Фракция Всероссийской политической партии ""ЕДИ...",12719.0,0.781
1,Альшевских Андрей Геннадьевич,"Фракция Всероссийской политической партии ""ЕДИ...",12703.0,0.780
2,Данчикова Галина Иннокентьевна,"Фракция Всероссийской политической партии ""ЕДИ...",12671.0,0.778
3,Марьяш Ирина Евгеньевна,"Фракция Всероссийской политической партии ""ЕДИ...",12654.0,0.777
4,Солнцева Светлана Юрьевна,"Фракция Всероссийской политической партии ""ЕДИ...",12654.0,0.777
5,Окунева Ольга Владимировна,"Фракция Всероссийской политической партии ""ЕДИ...",12654.0,0.777
6,Мельник Владимир Иванович,"Фракция Всероссийской политической партии ""ЕДИ...",12638.0,0.776
7,Канаев Алексей Валерианович,"Фракция Всероссийской политической партии ""ЕДИ...",12622.0,0.775
8,Боярский Сергей Михайлович,"Фракция Всероссийской политической партии ""ЕДИ...",12622.0,0.775
9,Ишсарин Рамзил Рафаилович,"Фракция Всероссийской политической партии ""ЕДИ...",12605.0,0.774


Обратный топ-10 депутатов, которые чаще голосуют "против". Тут картина совершенно другая -- сплошь системная оппозиция в лице КПРФ. 

Интересно, что даже у основных несогласных депутатов процент голосов "против" всё равно очень маленький -- меньше 5% от общего числа голосов.  

In [15]:
# топ-10 депутатов, которые больше голосовали "против"
top_no_dep = dep_full_data.groupby(['name', 'party'])[['voting_no_count', 'voting_no']].sum().sort_values(by='voting_no_count', ascending=False).reset_index()
top_no_dep.columns = ['имя_депутата', 'партия', 'всего_голосов_против', 'процент_голосов_против']
top_no_dep.head(10)

,имя_депутата,партия,всего_голосов_против,процент_голосов_против
0,Куринный Алексей Владимирович,"Фракция Политической партии ""КОММУНИСТИЧЕСКАЯ ...",765.0,0.047
1,Осадчий Николай Иванович,"Фракция Политической партии ""КОММУНИСТИЧЕСКАЯ ...",728.0,0.045
2,Коломейцев Николай Васильевич,"Фракция Политической партии ""КОММУНИСТИЧЕСКАЯ ...",717.0,0.044
3,Ганзя Вера Анатольевна,"Фракция Политической партии ""КОММУНИСТИЧЕСКАЯ ...",717.0,0.044
4,Арефьев Николай Васильевич,"Фракция Политической партии ""КОММУНИСТИЧЕСКАЯ ...",717.0,0.044
5,Казанков Сергей Иванович,"Фракция Политической партии ""КОММУНИСТИЧЕСКАЯ ...",700.0,0.043
6,Кравец Александр Алексеевич,"Фракция Политической партии ""КОММУНИСТИЧЕСКАЯ ...",700.0,0.043
7,Лебедев Олег Александрович,"Фракция Политической партии ""КОММУНИСТИЧЕСКАЯ ...",684.0,0.042
8,Парфенов Денис Андреевич,"Фракция Политической партии ""КОММУНИСТИЧЕСКАЯ ...",684.0,0.042
9,Поздняков Владимир Георгиевич,"Фракция Политической партии ""КОММУНИСТИЧЕСКАЯ ...",684.0,0.042


И вот тут становится понятно почему процент "против" такой низкий. Большинство оппозиционных депутатов предпочитают не голосовать вовсе, чем голосовать "против". 

In [18]:
# процент не голосовавших
party_votings = dep_full_data.groupby(['party'])[['voting_yes_count', 'voting_no_count', 'did_not_vote_count', 'votings']].sum().sort_values(by='voting_yes_count', ascending=False)
pd.DataFrame(party_votings.did_not_vote_count / party_votings.votings * 100, columns=['процент_не_голосовавших']).sort_values(by='процент_не_голосовавших', ascending=False)

,процент_не_голосовавших
party,
Фракция Политической партии ЛДПР - Либерально-демократической партии России,48.892852
"Депутаты, не входящие во фракции",48.158876
"Фракция Политической партии ""КОММУНИСТИЧЕСКАЯ ПАРТИЯ РОССИЙСКОЙ ФЕДЕРАЦИИ""",41.778670
Фракция Политической партии СПРАВЕДЛИВАЯ РОССИЯ,40.765328
"Фракция Всероссийской политической партии ""ЕДИНАЯ РОССИЯ""",29.161248


In [ ]:
# процент голосовавших за
pd.DataFrame(party_votings.voting_yes_count / party_votings.votings * 100, columns=['процент "за"']).sort_values(by='процент "за"', ascending=False)

,"процент ""за"""
party,
"Фракция Всероссийской политической партии ""ЕДИНАЯ РОССИЯ""",70.635910
Фракция Политической партии СПРАВЕДЛИВАЯ РОССИЯ,55.265822
"Фракция Политической партии ""КОММУНИСТИЧЕСКАЯ ПАРТИЯ РОССИЙСКОЙ ФЕДЕРАЦИИ""",53.980335
"Депутаты, не входящие во фракции",50.392876
Фракция Политической партии ЛДПР - Либерально-демократической партии России,49.313644


In [ ]:
# процент голосовавших против
pd.DataFrame(party_votings.voting_no_count / party_votings.votings * 100, columns=['процент "против"']).sort_values(by='процент "против"', ascending=False)

,"процент ""против"""
party,
"Фракция Политической партии ""КОММУНИСТИЧЕСКАЯ ПАРТИЯ РОССИЙСКОЙ ФЕДЕРАЦИИ""",3.921832
Фракция Политической партии СПРАВЕДЛИВАЯ РОССИЯ,3.515516
Фракция Политической партии ЛДПР - Либерально-демократической партии России,1.429266
"Депутаты, не входящие во фракции",1.050750
"Фракция Всероссийской политической партии ""ЕДИНАЯ РОССИЯ""",0.172220


### Скрейпинг данных

In [ ]:
search_bar = browser.find_element(By.XPATH, '//*[@id="input-search-1"]')

In [ ]:
search_bar.click()
search_bar.send_keys('Валуев Николай Сергеевич')

In [ ]:
dep_data = [i.text for i in browser.find_elements(By.XPATH, '//div[@class="person__description__col"]')]

In [ ]:
browser.find_element(By.XPATH, '//*[@id="input-search-1"]')

In [ ]:
dep_link = browser.find_element(By.XPATH, '//a[@class="autocomplete__result__link link link--color"]').get_attribute('href') 

In [ ]:
dep_link

In [ ]:
browser.get(url_1)
WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="input-search-1"]'))).click()

In [ ]:
search_bar.send_keys('fsafsafs')
try:
    WebDriverWait(browser, 3).until(EC.element_to_be_clickable((By.XPATH, '//h4[@class="autocomplete__result__title"]')))
    dep_data = [i.text for i in browser.find_elements(By.XPATH, '//div[@class="person__description__col"]')]
    dep_data = [i.text for i in browser.find_elements(By.XPATH, '//div[@class="person__description__col"]')]

except TimeoutException:
    print('Элемент не загрузился')

In [ ]:
dep_names = pd.read_csv('dep_names.csv', header=None)

In [ ]:
deputy = []

In [ ]:
for name in dep_names.values:
    browser.get('http://duma.gov.ru/duma/deputies/')
    WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="input-search-1"]'))).click()
    search_bar = browser.find_element(By.XPATH, '//*[@id="input-search-1"]')
    search_bar.send_keys(name[0])
    try:
        dep_data = []
        dep_link = []
        WebDriverWait(browser, 3).until(EC.element_to_be_clickable((By.XPATH, '//h4[@class="autocomplete__result__title"]')))
        dep_data = [i.text for i in browser.find_elements(By.XPATH, '//div[@class="person__description__col"]')]
        dep_link = browser.find_element(By.XPATH, '//a[@class="autocomplete__result__link link link--color"]').get_attribute('href')
        deputy.append([name[0], dep_data, dep_link])
        print(name[0], 'собран')
    except TimeoutException:
        print('Элемент не загрузился')

In [ ]:
dep_data = pd.read_csv('dep_data.csv')

In [ ]:
dep_data.head()

In [ ]:
links = dep_data['Ссылка']

In [ ]:
new_url = 'http://vote.duma.gov.ru/?convocation=AAAAAAA7&deputy=99112936&sort=date_desc'

In [ ]:
deps_id = []

In [ ]:
for i in links:
    deps_id.append(i.split('/')[5])

In [ ]:
for i in dep_absence_data:
    print(i[0])

In [ ]:
for i in dep_absence_data:
    print(i[2])

In [ ]:
dep_absence_data = []
for i in deps_id:
    try:
        new_url = f'http://vote.duma.gov.ru/?convocation=AAAAAAA7&deputy={i}&sort=date_desc'
        browser.get(new_url)
        WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.XPATH, '//div[@class="link-statis"]')))
        browser.find_element(By.XPATH, '//div[@class="link-statis"]').find_element_by_tag_name('span').click()
        time.sleep(2)
        WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.XPATH, '//div[@class="s-head"]')))
        presence_on_meeting = browser.find_element(By.XPATH, '//div[@class="pers-p"]').text.split('\n')[0]
        absence_on_meeting = browser.find_element(By.XPATH, '//div[@class="pers-m"]').text.split('\n')
        dep_name = browser.find_element(By.XPATH, '//div[@class="deputies"]').find_element_by_tag_name('span').text
        if len(absence_on_meeting) == 1:
            new_abs = 360 - int(presence_on_meeting)
            print(dep_name)
            print(presence_on_meeting)
            print(new_abs)
            dep_absence_data.append([dep_name, presence_on_meeting, new_abs])
        else:
            print(dep_name)
            print(presence_on_meeting)
            print(absence_on_meeting[0])
            dep_absence_data.append([dep_name, presence_on_meeting, absence_on_meeting[0]])
    except TimeoutException:
        print('Time is over')

In [ ]:
browser.get('http://vote.duma.gov.ru/?convocation=AAAAAAA7&deputy=99111804&sort=date_desc')

In [ ]:
WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.XPATH, '//span[@class="act"]'))).click()

In [ ]:
browser.find_element(By.XPATH, '//div[@class="link-statis"]').find_element_by_tag_name('span').click()

In [ ]:
amount_of_meetings = s_head.find_element_by_tag_name('span').text.split(':')[1]

In [ ]:
not_presence

In [ ]:
dep_1 = requests.get('http://duma.gov.ru/duma/persons/99111780/')
soup = BeautifulSoup(dep_1.text, 'lxml')

In [ ]:
soup.find('div', 'person__description__grid').find('a', 'link link--underline person__description__link').text

In [ ]:
soup.find('div', 'person__description__grid').find_all('div', 'person__description__col')

## Извлекаем всех действующих депутатов

In [ ]:
duma = requests.get('http://duma.gov.ru/duma/deputies/7/')
soup = BeautifulSoup(duma.text, 'lxml')
deputy_names = soup.find_all('h2', 'person__title person__title--s')
names = []
links = []
for tag in deputy_names:
    names.append(tag.get_text(separator=" ").strip())
    links.append('http://duma.gov.ru' + tag.a['href'])

In [ ]:
len(names)

443

In [ ]:
fractions = []
regions = []
laws = []
apsences = []
count = 0
for li in links:
    req = requests.get(li)
    soup = BeautifulSoup(req.text, 'lxml')
    fraction = soup.find('a', 'link link--underline person__description__link').text
    region = soup.find('a', 'link link--underline person__description__link').findNext('div').findNext('div').text
    law = soup.find('a', 'nav__link link--external').findNext('a')['href']
    apsence = soup.find('a', 'nav__link link--external').findNext('a').findNext('a')['href']
    fractions.append(fraction)
    regions.append(region)
    laws.append(law)
    apsences.append(apsence)
    count+=1

In [ ]:
dep_new_data = pd.DataFrame({
    'dep_links': links,
    'fractions': fractions,
    'laws_link': laws,
    'apsences_link': apsences
})

In [ ]:
dep_new_data.head()

,dep_links,fractions,laws_link,apsences_link
0,http://duma.gov.ru/duma/persons/99112936/,«ЕДИНАЯ РОССИЯ»,http://sozd.duma.gov.ru/oz?cond%5BThematicBloc...,http://vote.duma.gov.ru/?convocation=AAAAAAA7&...
1,http://duma.gov.ru/duma/persons/99112712/,«ЕДИНАЯ РОССИЯ»,http://sozd.duma.gov.ru/oz?cond%5BThematicBloc...,http://vote.duma.gov.ru/?convocation=AAAAAAA7&...
2,http://duma.gov.ru/duma/persons/99109653/,«ЕДИНАЯ РОССИЯ»,http://sozd.duma.gov.ru/oz?cond%5BThematicBloc...,http://vote.duma.gov.ru/?convocation=AAAAAAA7&...
3,http://duma.gov.ru/duma/persons/99100784/,СПРАВЕДЛИВАЯ РОССИЯ,http://sozd.duma.gov.ru/oz?cond%5BThematicBloc...,http://vote.duma.gov.ru/?convocation=AAAAAAA7&...
4,http://duma.gov.ru/duma/persons/99111778/,«ЕДИНАЯ РОССИЯ»,http://sozd.duma.gov.ru/oz?cond%5BThematicBloc...,http://vote.duma.gov.ru/?convocation=AAAAAAA7&...


In [ ]:
dep_1 = requests.get('http://duma.gov.ru/duma/persons/99112936/')
soup = BeautifulSoup(dep_1.text, 'lxml')

In [ ]:
for i in enumerate(fractions):
    if i[1] != '«ЕДИНАЯ РОССИЯ»' and i[1] !='СПРАВЕДЛИВАЯ РОССИЯ' and i[1] != 'ЛДПР' and i[1] != 'КПРФ':
        print(i)

In [ ]:
meetings = []
presence_percents = []
absence_precents = []
dep_names = []
dep_parties = []
voting_totals = []
voting_yeses = []
voting_noes = []
voting_abstaines = []
not_votes = []
missing_request = []

browser = webdriver.Chrome(executable_path=r'chromedriver.exe')

for li in dep_new_data.apsences_link:
    try:
        browser.get(li)
        WebDriverWait(browser, 3).until(EC.element_to_be_clickable((By.XPATH, '//div[@class="link-statis"]'))).click()
        time.sleep(3)
        total_meet = browser.find_element(By.XPATH, '//div[@class="s-head"]').find_element_by_tag_name('span').text # сколько заседаний всего
        presence_percent = browser.find_element(By.XPATH, '//div[@class="pers-p"]').find_elements_by_tag_name('span')[1].text # сколько заседаний посетил
        absence_precent = browser.find_element(By.XPATH, '//div[@class="pers-m"]').find_elements_by_tag_name('span')[1].text # сколько заседаний пропустил
        dep_name = browser.find_element(By.XPATH, '//div[@class="deputies"]').find_element_by_tag_name('span').text # имя депутата
        dep_party = browser.find_element(By.XPATH, '//div[@class="deputies"]').find_element_by_tag_name('a').text # партия
        voting_total = browser.find_element(By.XPATH, '//div[@class="statis-r"]').find_element_by_tag_name('div').find_element_by_tag_name('span').text # сколько голосований
        voting_yes = browser.find_element(By.XPATH, '//div[@class="ep-green"]').find_elements_by_tag_name('span')[2].text # голосования за
        voting_no = browser.find_element(By.XPATH, '//div[@class="ep-red"]').find_elements_by_tag_name('span')[2].text # голосования против
        voting_abstained = browser.find_element(By.XPATH, '//div[@class="ep-red"]').find_elements_by_tag_name('span')[2].text # голосования воздержался
        not_voted = browser.find_element(By.XPATH, '//div[@class="ep-gray"]').find_elements_by_tag_name('span')[2].text # не голосовал
        
        meetings.append(total_meet)
        presence_percents.append(presence_percent)
        absence_precents.append(absence_precent)
        dep_names.append(dep_name)
        dep_parties.append(dep_party)
        voting_totals.append(voting_total)
        voting_yeses.append(voting_yes)
        voting_noes.append(voting_no)
        voting_abstaines.append(voting_abstained)
        not_votes.append(not_voted)
        
        print(dep_name)
    except NoSuchElementException:
        missing_request.append(li)
        print('Страница не загрузилась')
    except TimeoutException:
        missing_request.append(li)
        print('Страница не загрузилась')

In [ ]:
dep_full_data = pd.DataFrame({
    'meetings':meetings,
    'percent_of_presence':presence_percents,
    'percent_of_absence':absence_precents,
    'name':dep_names,
    'party':dep_parties,
    'votings':voting_totals,
    'voting_yes':voting_yeses,
    'voting_no':voting_noes,
    'voting_abstained':voting_abstaines,
    'did_not_vote':not_votes,
})

In [ ]:
dep_full_data.to_csv('dep_full_data.csv')

In [ ]:
dep_full_data = pd.read_csv('dep_full_data.csv')
dep_full_data = dep_full_data.drop(columns='Unnamed: 0')

In [ ]:
dep_full_data.votings = dep_full_data.votings.apply(lambda x: x.split(": ")[1])

In [ ]:
dep_full_data.meetings = dep_full_data.meetings.apply(lambda x: x.split(": ")[1])

In [ ]:
dep_full_data[['meetings', 'votings']] = dep_full_data[['meetings', 'votings']].astype(int)

In [ ]:
dep_full_data.percent_of_presence = dep_full_data.percent_of_presence.str.strip('%')
dep_full_data.percent_of_absence = dep_full_data.percent_of_absence.str.strip('%')
dep_full_data.voting_yes = dep_full_data.voting_yes.str.strip('%')
dep_full_data.voting_no = dep_full_data.voting_no.str.strip('%')
dep_full_data.voting_abstained = dep_full_data.voting_abstained.str.strip('%')
dep_full_data.did_not_vote = dep_full_data.did_not_vote.str.strip('%')

In [ ]:
dep_full_data.percent_of_presence = dep_full_data.percent_of_presence.str.strip(' ')
dep_full_data.percent_of_absence = dep_full_data.percent_of_absence.str.strip(' ')
dep_full_data.voting_yes = dep_full_data.voting_yes.str.strip(' ')
dep_full_data.voting_no = dep_full_data.voting_no.str.strip(' ')
dep_full_data.voting_abstained = dep_full_data.voting_abstained.str.strip(' ')
dep_full_data.did_not_vote = dep_full_data.did_not_vote.str.strip(' ')

In [ ]:
dep_full_data[['percent_of_presence', 'percent_of_absence', 'voting_yes', 'voting_no', 'voting_abstained', 'did_not_vote']] = dep_full_data[['percent_of_presence', 'percent_of_absence', 'voting_yes', 'voting_no', 'voting_abstained', 'did_not_vote']].astype(float)

In [ ]:
dep_full_data[['percent_of_presence', 'percent_of_absence', 'voting_yes', 'voting_no', 'voting_abstained', 'did_not_vote']] = dep_full_data[['percent_of_presence', 'percent_of_absence', 'voting_yes', 'voting_no', 'voting_abstained', 'did_not_vote']].apply(lambda x: x / 100)

In [ ]:
dep_full_data['total_presence'] = round(dep_full_data.meetings * dep_full_data.percent_of_presence)
dep_full_data['total_absence'] = round(dep_full_data.meetings * dep_full_data.percent_of_absence)
dep_full_data['voting_yes_count'] = round(dep_full_data.votings * dep_full_data.voting_yes)
dep_full_data['voting_no_count'] = round(dep_full_data.votings * dep_full_data.voting_no)
dep_full_data['voting_abstained_count'] = round(dep_full_data.votings * dep_full_data.voting_abstained)
dep_full_data['did_not_vote_count'] = round(dep_full_data.votings * dep_full_data.did_not_vote)

In [ ]:
dep_full_data.dep_link[0]

'http://vote.duma.gov.ru/?convocation=AAAAAAA7&deputy=99112936&sort=date_desc'